In [ ]:
!git clone https://github.com/youwuyou/AISE.git

Cloning into 'AISE'...
remote: Enumerating objects: 1265, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 1265 (delta 59), reused 83 (delta 31), pack-reused 1143 (from 2)
Receiving objects: 100% (1265/1265), 324.43 MiB | 14.06 MiB/s, done.
Resolving deltas: 100% (624/624), done.
Filtering content: 100% (10/10), 90.12 MiB | 24.12 MiB/s, done.
Encountered 10 file(s) that should have been pointers, but weren't:
	project_1/checkpoints/all2all/fno_m30_w32_d2_lr0.001_20250102_152429/model.pth
	project_1/checkpoints/onetoall/fno_m30_w32_d2_lr0.001_20250102_151601/model.pth
	project_1/checkpoints/onetoone/fno_m30_w16_d2_lr0.001_20250102_145624/model.pth
	project_1/data/test_sol.npy
	project_1/data/test_sol_OOD.npy
	project_1/data/test_sol_res_128.npy
	project_1/data/test_sol_res_32.npy
	project_1/data/test_sol_res_64.npy
	project_1/data/test_sol_res_96.npy
	project_1/data/train_sol.npy


In [ ]:
%cd AISE/project_1
%ls

/content/AISE/project_1
checkpoints/  evaluate.py  results/                utils.py
data/         fno.py       train_fno.py            visualization.py
dataset.py    README.md    train_fno_with_time.py


In [ ]:
import torch
import numpy as np

from torch.utils.data import Dataset, DataLoader

class OneToOne(Dataset):
    def __init__(self,
                which,
                training_samples=64,
                data_path="data/train_sol.npy",
                lx=1.0,
                dt=0.25,
                start_idx=0,
                end_idx=4,
                device='cuda'):
        # dataset = torch.from_numpy(np.load(data_path))
        dataset = torch.from_numpy(np.load(data_path)).type(torch.float32)

        if device == 'cuda':
            dataset = dataset.type(torch.float32).to(device)

        if which == "training":
            self.data = dataset[:training_samples]
        elif which == "validation":
            self.data = dataset[training_samples:]
        elif which == "testing":
            self.data = dataset
        else:
            raise ValueError("Dataset must be 'training', 'validation' or `testing`")

        self.length = len(self.data)
        self.dt = dt
        self.nt = self.data.shape[1]
        num_timesteps = self.nt - 1

        # Stack all available timesteps
        self.u = torch.stack([self.data[:, i, :] for i in range(self.nt)], dim=1)

        # Calculate time derivatives based on available timesteps
        # Multiple timesteps - use central differences where possible
        self.v = []
        for i in range(self.nt - 1):
            # print(f"current i: {i}")
            if i == 0:
                # Set initial velocity to zero
                deriv = torch.zeros_like(self.u[:, 0])
            elif i == self.nt:
                # Backward difference for last point
                deriv = (self.u[:, -1] - self.u[:, -2]) / self.dt
            else:
                # Central difference for interior points
                deriv = (self.u[:, i+1] - self.u[:, i-1]) / (2 * self.dt)
            self.v.append(deriv)
        self.v = torch.stack(self.v, dim=1)

        # Domain setup
        self.lx = lx
        self.nx = self.data.shape[-1]
        self.x_grid = torch.linspace(0, self.lx, self.nx).to(device)

        # Adjust indices if they exceed available timesteps
        assert(end_idx >= start_idx)
        self.start_idx = start_idx
        self.end_idx = end_idx
        self.u_start = self.u[:, self.start_idx]
        self.v_start = self.v[:, self.start_idx]
        self.u_end = self.u[:, self.end_idx]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        u_start = self.u_start[idx]
        v_start = self.v_start[idx]
        dt = torch.full_like(u_start, self.dt * (self.end_idx - self.start_idx), device=u_start.device)
        x_grid = self.x_grid.to(u_start.device)
        input_data = torch.stack((u_start, v_start, x_grid, dt), dim=-1)
        output_data = self.u_end[idx].unsqueeze(-1)
        return input_data, output_data


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SpectralConv1d(nn.Module):
    """
    The FNO1d uses SpectralConv1d as its crucial part,
        - implements the Fourier integral operator in a layer
            F⁻¹(R⁽ˡ⁾∘F)(u)
        - uses FFT, linear transform, and inverse FFT, applicable to equidistant mesh
    """
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()
        if not isinstance(modes1, int) or modes1 <= 0:
            raise ValueError(f"modes1 must be a positive integer, got {modes1}")

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1  # Number of Fourier modes to multiply, at most floor(N/2) + 1

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, dtype=torch.cfloat))

    def compl_mul1d(self, input, weights):
        """
        Complex multiplication in 1D
        (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)
        """
        return torch.einsum("bix,iox->box", input, weights)

    def forward(self, x):
        """
            1) Compute Fourier coefficients
            2) Multiply relevant Fourier modes
            3) Transform the data to physical space
            HINT: Use torch.fft library torch.fft.rfft
        """
        batchsize = x.shape[0]

        # Compute Fourier coefficients up to factor of e^(- something constant)
        x_ft = torch.fft.rfft(x)

        # Use min to limit modes to what's available
        effective_modes = min(self.modes1, x.size(-1) // 2 + 1)

        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-1)//2 + 1,
                        device=x.device, dtype=torch.cfloat)
        out_ft[:, :, :effective_modes] = self.compl_mul1d(x_ft[:, :, :effective_modes],
                                                    self.weights1[:,:,:effective_modes])

        x = torch.fft.irfft(out_ft, n=x.size(-1))
        return x

class FNO1d(nn.Module):
    def __init__(self, modes, width, depth, device="cuda", nfun=1, padding_frac=1/4, time_dependent=False):
        """
        The overall network 𝒢_θ(u). It contains [depth] layers of the Fourier layers.
        Each layer implements:
        u_(l+1) = σ(K^(l)(u_l) + W^(l)u_l + b^(l))
        where:
        - K^(l) is the Fourier integral operator F⁻¹(R⁽ˡ⁾∘F)
        - W^(l) is the residual connection
        - b^(l) is the bias term

        Complete architecture:
        1. Lift the input to the desired channel dimension by P (self.fc0)
        2. Apply [depth] layers of Fourier integral operators with residual connections
        3. Project from the channel space to the output space by Q (self.fc1 and self.fc2)

        input: the solution of the initial condition and location (a(x), x)
        input shape: (batchsize, x=s, c=2)
        output: the solution of a later timestep
        output shape: (batchsize, x=s, c=1)
        """
        super(FNO1d, self).__init__()

        self.modes = modes
        self.width = width
        self.depth = depth
        self.padding_frac = padding_frac
        self.time_dependent = time_dependent

        self.fc0 = nn.Linear(nfun + 1, self.width) # +1 for x_grid

        # Fourier layers
        self.spectral_list = nn.ModuleList([
            SpectralConv1d(self.width, self.width, self.modes) for _ in range(self.depth)
        ])

        # Linear residual connections
        self.w_list = nn.ModuleList([
            nn.Linear(self.width, self.width, bias=False) for _ in range(self.depth)
        ])

        # Bias terms
        self.b_list = nn.ParameterList([
            nn.Parameter(torch.zeros(1, self.width, 1)) for _ in range(self.depth)
        ])


        # Projection layers
        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

        self.activation = nn.GELU()

        self.to(device)  # Move model to specified device

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x: Input tensor of shape [batch_size, sequence_length, channel]

        Returns:
            Output tensor of shape [batch_size, sequence_length, 1]
        """
        u_start = x[..., 0].unsqueeze(-1)
        v_start = x[..., 1].unsqueeze(-1)
        x_grid  = x[..., 2].unsqueeze(-1)

        # dt will not be used if time-dependent is not set True
        dt = x[..., 3].unsqueeze(-1)  # Keep as [batch]

        x = torch.cat((u_start, v_start, x_grid), dim=-1)
        x = self.fc0(x)
        x = x.permute(0, 2, 1)  # Now shape is [batch, width, sequence_length]

        # Apply padding
        x_padding = int(round(x.shape[-1] * self.padding_frac))
        x = F.pad(x, [0, x_padding])

        # Apply Fourier layers with residual connections and bias
        for i in range(self.depth):
            # Store input for residual connection
            x_input = x
            # Fourier integral operator K^(l)
            x1 = self.spectral_list[i](x)  # Shape: [batch, width, sequence_length]
            # Residual connection W^(l)
            x2 = self.w_list[i](x_input.transpose(1, 2)).transpose(1, 2)
            # Expand b to match the sequence length
            b_expanded = self.b_list[i].expand(-1, -1, x1.size(-1))

            # Combine with bias: K^(l) + W^(l) + b^(l)
            x = x1 + x2 + b_expanded

            # Apply time-conditional normalization if time-dependent
            if self.time_dependent:
                # tensor([[a], [b], [c], [d], [e]])
                dt = dt[:, 0].unsqueeze(-1)  # or dt[:, 0].view(-1, 1)
                x = self.film_list[i](x, dt)

            # Apply activation (except for last layer)
            if i != self.depth - 1:
                x = self.activation(x)

        # Remove padding
        x = x[..., :-x_padding]

        # Final projection layers Q
        x = x.permute(0, 2, 1)  # Back to [batch, sequence_length, width]
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x  # Shape should be [batch_size, sequence_length, 1]

    def print_size(self):
        nparams = 0
        nbytes = 0
        for param in self.parameters():
            nparams += param.numel()
            nbytes += param.data.element_size() * param.numel()
        print(f'Total number of model parameters: {nparams}')
        return nparams

In [ ]:
class GatedFNO1d(nn.Module):
    def __init__(self, modes, width, depth, device="cuda", nfun=1, padding_frac=1/4, time_dependent=False):
        super().__init__()

        self.modes = modes
        self.width = width
        self.depth = depth
        self.padding_frac = padding_frac
        self.time_dependent = time_dependent

        self.fc0 = nn.Linear(nfun + 1, self.width) # +1 for x_grid

        # Fourier layers
        self.value_spectral_list = nn.ModuleList([
            SpectralConv1d(self.width, self.width, self.modes) for _ in range(self.depth)
        ])
        self.gate_spectral_list = nn.ModuleList([
            SpectralConv1d(self.width, self.width, self.modes) for _ in range(self.depth)
        ])

        # Linear residual connections
        self.value_w_list = nn.ModuleList([
            nn.Linear(self.width, self.width, bias=False) for _ in range(self.depth)
        ])
        self.gate_w_list = nn.ModuleList([
            nn.Linear(self.width, self.width, bias=False) for _ in range(self.depth)
        ])

        # Bias terms
        self.value_b_list = nn.ParameterList([
            nn.Parameter(torch.zeros(1, self.width, 1)) for _ in range(self.depth)
        ])
        self.gate_b_list = nn.ParameterList([
            nn.Parameter(torch.zeros(1, self.width, 1)) for _ in range(self.depth)
        ])


        # Projection layers
        self.value_fc1 = nn.Linear(self.width, 128)
        self.gate_fc1 = nn.Linear(self.width, 128)
        self.value_fc2 = nn.Linear(128, 1)


        self.to(device)  # Move model to specified device

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x: Input tensor of shape [batch_size, sequence_length, channel]

        Returns:
            Output tensor of shape [batch_size, sequence_length, 1]
        """
        u_start = x[..., 0].unsqueeze(-1)
        v_start = x[..., 1].unsqueeze(-1)
        x_grid  = x[..., 2].unsqueeze(-1)

        # dt will not be used if time-dependent is not set True
        dt = x[..., 3].unsqueeze(-1)  # Keep as [batch]

        x = torch.cat((u_start, v_start, x_grid), dim=-1)
        x = self.fc0(x)
        x = x.permute(0, 2, 1)  # Now shape is [batch, width, sequence_length]

        # Apply padding
        x_padding = int(round(x.shape[-1] * self.padding_frac))
        gate_x = F.pad(x, [0, x_padding])
        value_x = torch.ones_like(gate_x)


        # Apply Fourier layers with residual connections and bias
        for i in range(self.depth):
            # Value Network
            value_x1 = self.value_spectral_list[i](value_x)
            value_x2 = self.value_w_list[i](value_x.transpose(1, 2)).transpose(1, 2)
            value_b_expanded = self.value_b_list[i].expand(-1, -1, value_x1.size(-1))
            value_x = value_x1 + value_x2 + value_b_expanded

            #Gating Network
            gate_x1 = self.gate_spectral_list[i](gate_x)
            gate_x2 = self.gate_w_list[i](gate_x.transpose(1, 2)).transpose(1, 2)
            gate_b_expanded = self.gate_b_list[i].expand(-1, -1, gate_x1.size(-1))
            gate_x = gate_x1 + gate_x2 + gate_b_expanded

            if i != self.depth - 1:
               value_x = torch.sigmoid(gate_x) * value_x

        # Remove padding
        value_x = value_x[..., :-x_padding]
        gate_x = gate_x[..., :-x_padding]

        # Final projection layers Q
        value_x = value_x.permute(0, 2, 1)
        gate_x = gate_x.permute(0, 2, 1)  # Back to [batch, sequence_length, width]
        value_x = self.value_fc1(value_x)
        gate_x = self.gate_fc1(gate_x)
        x = value_x * torch.sigmoid(gate_x)
        x = self.value_fc2(x)
        return x  # Shape should be [batch_size, sequence_length, 1]

    def print_size(self):
        nparams = 0
        nbytes = 0
        for param in self.parameters():
            nparams += param.numel()
            nbytes += param.data.element_size() * param.numel()
        print(f'Total number of model parameters: {nparams}')
        return nparams

In [ ]:
import torch
import numpy as np
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset, Dataset
from utils import (
train_model,
get_experiment_name,
save_config
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU device:", torch.cuda.get_device_name(0))

Using device: cuda
CUDA available: True
GPU device: Tesla T4


In [ ]:
torch.manual_seed(0)
np.random.seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)

In [ ]:
model_config = {
    "depth": 2,
    "modes": 30,
    "width": 16,
    "nfun": 2,
    "time_dependent": False,
    "device": device
}

# Training configuration
data_mode = "onetoone"
training_config = {
    "data_mode": data_mode,
    'n_train': 64,
    'batch_size': 5,
    'learning_rate': 0.0005,
    'epochs': 1200,
    'step_size': 300,
    'gamma': 0.1,
    'patience': 80,
    'freq_print': 1,
    'device': device
}

# Combine configurations for experiment naming
naming_config = {**model_config, 'learning_rate': training_config['learning_rate']}

# Create experiment directory
experiment_name = get_experiment_name(naming_config)
checkpoint_dir = Path(f"checkpoints/{data_mode}") / experiment_name
checkpoint_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
config = {
    'model_config': model_config,
    'training_config': training_config
}
save_config(config, checkpoint_dir)

In [ ]:
batch_size = training_config['batch_size']
training_set = DataLoader(OneToOne("training"), batch_size=batch_size, shuffle=True)
testing_set  = DataLoader(OneToOne("validation"), batch_size=batch_size, shuffle=False)

# Initialize model with device
model = GatedFNO1d(**{k: v for k, v in model_config.items()})

In [ ]:
trained_model, training_history = train_model(
    model=model,
    training_set=training_set,
    testing_set=testing_set,
    config=training_config,
    checkpoint_dir=checkpoint_dir
)

print(f"Training completed. Gated Model saved in {checkpoint_dir}")
print(f"Best validation loss: {training_history['best_val_loss']:.6f} at epoch {training_history['best_epoch']}")

Epoch: 0, Train Loss: 0.031874, Validation Loss: 0.034051
Epoch: 1, Train Loss: 0.031276, Validation Loss: 0.033389
Epoch: 2, Train Loss: 0.030836, Validation Loss: 0.032500
Epoch: 3, Train Loss: 0.030179, Validation Loss: 0.031341
Epoch: 4, Train Loss: 0.029100, Validation Loss: 0.030345
Epoch: 5, Train Loss: 0.027499, Validation Loss: 0.027364
Epoch: 6, Train Loss: 0.025127, Validation Loss: 0.024558
Epoch: 7, Train Loss: 0.022599, Validation Loss: 0.021741
Epoch: 8, Train Loss: 0.020725, Validation Loss: 0.020447
Epoch: 9, Train Loss: 0.019442, Validation Loss: 0.019337
Epoch: 10, Train Loss: 0.017974, Validation Loss: 0.018054
Epoch: 11, Train Loss: 0.016637, Validation Loss: 0.016942
Epoch: 12, Train Loss: 0.015496, Validation Loss: 0.016228
Epoch: 13, Train Loss: 0.014721, Validation Loss: 0.015775
Epoch: 14, Train Loss: 0.014548, Validation Loss: 0.015653
Epoch: 15, Train Loss: 0.014387, Validation Loss: 0.015244
Epoch: 16, Train Loss: 0.014219, Validation Loss: 0.015172
Epoch: 

In [ ]:
trained_model, training_history = train_model(
    model=model,
    training_set=training_set,
    testing_set=testing_set,
    config=training_config,
    checkpoint_dir=checkpoint_dir
)

print(f"Training completed. Model saved in {checkpoint_dir}")
print(f"Best validation loss: {training_history['best_val_loss']:.6f} at epoch {training_history['best_epoch']}")

Epoch: 0, Train Loss: 0.031413, Validation Loss: 0.031788
Epoch: 1, Train Loss: 0.028967, Validation Loss: 0.028255
Epoch: 2, Train Loss: 0.025593, Validation Loss: 0.023365
Epoch: 3, Train Loss: 0.021810, Validation Loss: 0.020979
Epoch: 4, Train Loss: 0.019611, Validation Loss: 0.018133
Epoch: 5, Train Loss: 0.017027, Validation Loss: 0.016073
Epoch: 6, Train Loss: 0.014855, Validation Loss: 0.014894
Epoch: 7, Train Loss: 0.013317, Validation Loss: 0.013690
Epoch: 8, Train Loss: 0.012136, Validation Loss: 0.012008
Epoch: 9, Train Loss: 0.010304, Validation Loss: 0.009497
Epoch: 10, Train Loss: 0.007944, Validation Loss: 0.006859
Epoch: 11, Train Loss: 0.005328, Validation Loss: 0.004613
Epoch: 12, Train Loss: 0.003498, Validation Loss: 0.003184
Epoch: 13, Train Loss: 0.002638, Validation Loss: 0.002899
Epoch: 14, Train Loss: 0.002585, Validation Loss: 0.003017
Epoch: 15, Train Loss: 0.002447, Validation Loss: 0.002326
Epoch: 16, Train Loss: 0.002077, Validation Loss: 0.002447
Epoch: 

In [ ]:
import torch
import numpy as np
from pathlib import Path

from visualization import (
    plot_training_history,
    plot_resolution_comparison,
    plot_l2_error_by_resolution,
    plot_error_distributions,
    plot_ibvp_sol_heatmap,
    plot_model_errors_temporal
)
from utils import (
    print_bold,
    evaluate_direct,
    evaluate_autoregressive
)

res_dir = Path('results')
res_dir.mkdir(exist_ok=True)
time_res_dir = res_dir / 'time'
time_res_dir.mkdir(exist_ok=True)

In [ ]:
import torch
import numpy as np
import json
from datetime import datetime
from pathlib import Path
from torch.utils.data import DataLoader

def load_model(checkpoint_dir: str) -> torch.nn.Module:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    checkpoint_dir = Path(checkpoint_dir)

    with open(checkpoint_dir / 'training_config.json', 'r') as f:
        config_dict = json.load(f)

    model_config = config_dict['model_config']
    model_args = {k: v for k, v in model_config.items()}
    model = GatedFNO1d(**model_args)

    model.load_state_dict(torch.load(checkpoint_dir / 'model.pth', weights_only=True))
    model = model.to(device)
    model.eval()
    return model

In [ ]:
# Time-independent evaluation
data_mode = "onetoone"
fno_folders = sorted(Path(f'checkpoints/{data_mode}').glob('fno_*'),
                    key=lambda d: d.stat().st_mtime)

In [ ]:
fno_folders

[PosixPath('checkpoints/onetoone/fno_m30_w16_d2_lr0.001_20250102_145624'),
 PosixPath('checkpoints/onetoone/fno_m30_w16_d2_lr0.001_20250411_183037')]

In [ ]:
model = load_model(fno_folders[-1])
print(f"Loading Custom FNO from: {fno_folders[-1]}")

print("Plotting training history...")
plot_training_history(fno_folders[-1])

Loading Custom FNO from: checkpoints/onetoone/fno_m30_w16_d2_lr0.001_20250411_183037
Plotting training history...


In [ ]:
model = load_model(fno_folders[-1])
print(f"Loading Custom FNO from: {fno_folders[-1]}")

print("Plotting training history...")
plot_training_history(fno_folders[-1])

Loading Custom FNO from: checkpoints/onetoone/fno_m30_w16_d2_lr0.001_20250411_170218
Plotting training history...


In [ ]:
print_bold("Task 1: Evaluating FNO model from One-to-One training on standard test set...")
result, test_data = evaluate_direct(model, "data/test_sol.npy")
print(f"Resolution: {test_data[0].shape[0]}")
print(f"Average Relative L2 Error Over {test_data[0].shape[0]} Testing Trajectories (resolution {test_data[0].shape[1]}):")
print(f"Custom FNO: {result['error']:.2f}%")


Task 1: Evaluating FNO model from One-to-One training on standard test set...
Resolution: 128
Average Relative L2 Error Over 128 Testing Trajectories (resolution 4):
Custom FNO: 9.18%


In [ ]:
print_bold("Task 1: Evaluating FNO model from One-to-One training on standard test set...")
result, test_data = evaluate_direct(model, "data/test_sol.npy")
print(f"Resolution: {test_data[0].shape[0]}")
print(f"Average Relative L2 Error Over {test_data[0].shape[0]} Testing Trajectories (resolution {test_data[0].shape[1]}):")
print(f"Custom FNO: {result['error']:.2f}%")


Task 1: Evaluating FNO model from One-to-One training on standard test set...
Resolution: 128
Average Relative L2 Error Over 128 Testing Trajectories (resolution 4):
Custom FNO: 5.21%


In [ ]:
print_bold("Task 2: Testing on different resolutions:")
resolutions = [32, 64, 96, 128]
resolution_results = {'Custom FNO': {'errors': [], 'predictions': {}, 'abs_errors': []}}

for res in resolutions:
    print(f"Resolution: {res}")
    result, test_data = evaluate_direct(model, f"data/test_sol_res_{res}.npy", end_idx=1)

    print(f"Average Relative L2 Error Over {test_data[0].shape[0]} Testing Trajectories (resolution {test_data[0].shape[1]}):")
    print(f"Custom FNO: {result['error']:.2f}%")
    print("-" * 50)
    resolution_results['Custom FNO']['errors'].append(result['error'])
    resolution_results['Custom FNO']['predictions'][res] = result['predictions']

resolution_data = {}
for res in resolutions:
    dataset = OneToOne(
        which="testing",
        data_path=f"data/test_sol_res_{res}.npy",
        start_idx=0,
        end_idx=1
    )

    input_data = torch.stack((
        dataset.u_start,
        dataset.v_start,
        dataset.x_grid.repeat(len(dataset.u_start), 1),
        torch.full_like(dataset.u_start, dataset.dt)
    ), dim=-1)

    resolution_data[res] = {'custom': (input_data, dataset.u_end)}

plot_resolution_comparison(model, resolution_data, resolution_results, save_dir=res_dir)
plot_l2_error_by_resolution(resolution_results, resolutions, save_dir=res_dir)


Task 2: Testing on different resolutions:
Resolution: 32
Average Relative L2 Error Over 128 Testing Trajectories (resolution 4):
Custom FNO: 22.66%
--------------------------------------------------
Resolution: 64
Average Relative L2 Error Over 128 Testing Trajectories (resolution 4):
Custom FNO: 8.62%
--------------------------------------------------
Resolution: 96
Average Relative L2 Error Over 128 Testing Trajectories (resolution 4):
Custom FNO: 9.47%
--------------------------------------------------
Resolution: 128
Average Relative L2 Error Over 128 Testing Trajectories (resolution 4):
Custom FNO: 9.74%
--------------------------------------------------


In [ ]:
print_bold("Task 2: Testing on different resolutions:")
resolutions = [32, 64, 96, 128]
resolution_results = {'Custom FNO': {'errors': [], 'predictions': {}, 'abs_errors': []}}

for res in resolutions:
    print(f"Resolution: {res}")
    result, test_data = evaluate_direct(model, f"data/test_sol_res_{res}.npy", end_idx=1)

    print(f"Average Relative L2 Error Over {test_data[0].shape[0]} Testing Trajectories (resolution {test_data[0].shape[1]}):")
    print(f"Custom FNO: {result['error']:.2f}%")
    print("-" * 50)
    resolution_results['Custom FNO']['errors'].append(result['error'])
    resolution_results['Custom FNO']['predictions'][res] = result['predictions']

resolution_data = {}
for res in resolutions:
    dataset = OneToOne(
        which="testing",
        data_path=f"data/test_sol_res_{res}.npy",
        start_idx=0,
        end_idx=1
    )

    input_data = torch.stack((
        dataset.u_start,
        dataset.v_start,
        dataset.x_grid.repeat(len(dataset.u_start), 1),
        torch.full_like(dataset.u_start, dataset.dt)
    ), dim=-1)

    resolution_data[res] = {'custom': (input_data, dataset.u_end)}

plot_resolution_comparison(model, resolution_data, resolution_results, save_dir=res_dir)
plot_l2_error_by_resolution(resolution_results, resolutions, save_dir=res_dir)


Task 2: Testing on different resolutions:
Resolution: 32
Average Relative L2 Error Over 128 Testing Trajectories (resolution 4):
Custom FNO: 20.07%
--------------------------------------------------
Resolution: 64
Average Relative L2 Error Over 128 Testing Trajectories (resolution 4):
Custom FNO: 4.99%
--------------------------------------------------
Resolution: 96
Average Relative L2 Error Over 128 Testing Trajectories (resolution 4):
Custom FNO: 5.92%
--------------------------------------------------
Resolution: 128
Average Relative L2 Error Over 128 Testing Trajectories (resolution 4):
Custom FNO: 6.39%
--------------------------------------------------


In [ ]:
print_bold("Task 3: Testing on OOD dataset:")

in_result, in_data = evaluate_direct(model, "data/test_sol.npy")
ood_result, ood_data = evaluate_direct(model, "data/test_sol_OOD.npy", end_idx=1)

print(f"In-Distribution - Average Relative L2 Error Over {in_data[0].shape[0]} Testing Trajectories:")
print(f"Custom FNO: {in_result['error']:.2f}%")
print("-" * 50)

print(f"Out-of-Distribution - Average Relative L2 Error Over {ood_data[0].shape[0]} Testing Trajectories:")
print(f"Custom FNO: {ood_result['error']:.2f}%")

in_dist_results = {'Custom FNO': in_result}
ood_results = {'Custom FNO': ood_result}
plot_error_distributions(in_dist_results, ood_results, save_path=res_dir / 'error_distributions.png')


Task 3: Testing on OOD dataset:
In-Distribution - Average Relative L2 Error Over 128 Testing Trajectories:
Custom FNO: 9.18%
--------------------------------------------------
Out-of-Distribution - Average Relative L2 Error Over 128 Testing Trajectories:
Custom FNO: 13.94%


In [ ]:
print_bold("Task 3: Testing on OOD dataset:")

in_result, in_data = evaluate_direct(model, "data/test_sol.npy")
ood_result, ood_data = evaluate_direct(model, "data/test_sol_OOD.npy", end_idx=1)

print(f"In-Distribution - Average Relative L2 Error Over {in_data[0].shape[0]} Testing Trajectories:")
print(f"Custom FNO: {in_result['error']:.2f}%")
print("-" * 50)

print(f"Out-of-Distribution - Average Relative L2 Error Over {ood_data[0].shape[0]} Testing Trajectories:")
print(f"Custom FNO: {ood_result['error']:.2f}%")

in_dist_results = {'Custom FNO': in_result}
ood_results = {'Custom FNO': ood_result}
plot_error_distributions(in_dist_results, ood_results, save_path=res_dir / 'error_distributions.png')


Task 3: Testing on OOD dataset:
In-Distribution - Average Relative L2 Error Over 128 Testing Trajectories:
Custom FNO: 5.21%
--------------------------------------------------
Out-of-Distribution - Average Relative L2 Error Over 128 Testing Trajectories:
Custom FNO: 9.77%


In [ ]:

task1_results = task1_evaluation(model, res_dir)
task2_results = task2_evaluation(model, res_dir)
task3_results = task3_evaluation(model, res_dir)